# Assignment Lending Club Case Study
## By: Edwin Mathew
## Date : 10/08/2022

## Business Understanding/Objectives

We got a request from an online loan marketplace which specialize in Facilitating Personal Loans, business loans and financing of medical procedure. The company like to do a Exploratory Credit Risk Analytics on the lending data available to them, main objective of this task is to understand the 'risky' applicants that can cause Credit loss due to default. If the analysis can be used to identify the risky loan applicants, then such loans can be reduced therby cutting down the amount of credit loss due to defaults.

Aim of this EDA is to find the driving factors behind the loan default, variables which are strong indicators will be presented. 


In [ ]:
#import the required libraries
import numpy as np
from numpy import percentile
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import plotly.express as px



# this is to see more columns during analysis  
pd.options.display.max_columns = None
# this is to see more rows during analysis 
pd.options.display.max_rows = None

#### Read csv file and store it in dataframe then use this to find the shape, data structure and observe the data structure

In [ ]:
loandata = pd.read_csv(r'loan.csv')
print(loandata.shape)
loandata.head(5)

data.csv contains 39717 rows and 111 coloumns.
### Next step is Cleaning the Data
1. Check the data type to find out the generic view of the data 

In [ ]:
loandata.info()

2. Find out how many coloumns have null values and what percentage to see the data integrity

In [ ]:
loandata[loandata.columns[loandata.isnull().any()]].isnull().sum() * 100 / loandata.shape[0]

3. In the output most of the columns have more than 60% of data missing these can be dropped without causing any issues to the structure, desc column only had 32% data missing but the information has been grabbed by purpose and title columns so this can be removed as well. 
4. lets look at the rest of the columns to understand a bit further (additional 10 coloumns) and do missing data imputation 


In [ ]:
loandata = loandata[loandata.columns[((loandata.isnull().sum())/len(loandata)) < 0.3]]
loandata[loandata.columns[loandata.isnull().any()]].isnull().sum() * 100 / loandata.shape[0]


5. Now implute mode values in for the categorial data in here such as emp_title and title
  . Both last payment date and last credit pull date needs to split into two coloumns and fill the NaNs
  . collections_12_mths_ex_med, chargeoff_within_12_mths and tax_liens only contain 0 and Nan (this does not show 
    any crucial information. title is a full description of purpose which is cateogrised so we can drop this as         well)
  . mode of the emp title and pub_rec_bankruptcies will be used to fill NaN's
  . Revol_util % is removed converted to float and then used median value value to replace all the Nans here

In [ ]:
loandata['pub_rec_bankruptcies'].fillna(loandata['pub_rec_bankruptcies'].mode()[0], inplace=True)
loandata = loandata.drop(['collections_12_mths_ex_med', 'chargeoff_within_12_mths','tax_liens','title'],axis = 1)

#removing % sign and converting the revol util into float then finding median to replace the missing value
loandata['revol_util'] = loandata['revol_util'].str.replace(r'%', r'').astype('float')
loandata['revol_util'].fillna(loandata['revol_util'].median(), inplace=True)

#convert the 10+ to 10 years and less than 1 year to 0 year
loandata['emp_length'] = loandata['emp_length'].replace(['10+ years','< 1 year'],['10 years','0 years'])
loandata['emp_length'] = loandata['emp_length'].str.replace(r'years', r'')
loandata['emp_length'] = loandata['emp_length'].str.replace(r'year', r'')

# replace the missing value with the mode  emp length
loandata['emp_length'].fillna(loandata['emp_length'].mode()[0], inplace=True)

#split the last payment date into two seperate columns 'Month' and 'Year' then delete the existing column
loandata['last_pymnt_Month'], loandata['last_pymnt_Year'] = loandata['last_pymnt_d'].str.split('-', 1).str
loandata['last_pymnt_Month'].fillna(loandata['last_pymnt_Month'].mode()[0], inplace=True)
loandata['last_pymnt_Year'].fillna(loandata['last_pymnt_Year'].mode()[0], inplace=True)
loandata = loandata.drop(['last_pymnt_d'],axis = 1)

#split the earliest credit into two seperate columns 'Month' and 'Year' then delete the existing column
loandata['earliest_cr_Month'], loandata['earliest_cr_Year'] = loandata['earliest_cr_line'].str.split('-', 1).str
loandata = loandata.drop(['earliest_cr_line'],axis = 1)

#split the last credit pull date into two seperate columns 'Month' and 'Year' then delete the existing column
loandata['last_credit_pull_Month'], loandata['last_credit_pull_Year'] = loandata['last_credit_pull_d'].str.split('-', 1).str
loandata['last_credit_pull_Month'].fillna(loandata['last_credit_pull_Month'].mode()[0], inplace=True)
loandata['last_credit_pull_Year'].fillna(loandata['last_credit_pull_Year'].mode()[0], inplace=True)
loandata = loandata.drop(['last_credit_pull_d'],axis = 1)

#since there is no missing data in these column we can use the strptime function take out Year and Month

loandata['issue_Year'] = pd.to_datetime(loandata['issue_d'].apply(lambda x: datetime.strptime(x,'%b-%y'))).dt.strftime('%Y')
loandata['issue_Month'] = pd.to_datetime(loandata['issue_d'].apply(lambda x: datetime.strptime(x,'%b-%y'))).dt.strftime('%b')
loandata = loandata.drop(['issue_d'],axis = 1)


# Removing XX from from the end Zipcode since it does not hold any value
loandata['zip_code'] = loandata['zip_code'].str.replace(r'xx', r'').astype('str')


#Removing % from int rate and converting into a float number
loandata['int_rate'] = loandata['int_rate'].str.replace(r'%', r'').astype('float')

#find duplicates of ID and member ID columns to see if there is any trend or difference
print(len(loandata['id'])-len(loandata['id'].drop_duplicates()))
print(len(loandata['member_id'])-len(loandata['member_id'].drop_duplicates()))
# since there are no trend or duplication found these rows can be removed

loandata = loandata.drop(['id', 'member_id'],axis = 1)

#Although Emp Title is important there is no standardisation across the data
loandata = loandata.drop(['emp_title'],axis = 1)


In [ ]:
# the outliers of the annual income needs to be removed since this can throw off the data
# calculate interquartile range
q25, q75 = percentile(loandata['annual_inc'], 25), percentile(loandata['annual_inc'], 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
#find out the outlier cutoff
cutoff = iqr * 1.5
lowerc, upperc = q25 - cutoff, q75 + cutoff
print (cutoff,lowerc,upperc)

# identify and remove outliers
loandata_up = loandata.loc[(loandata['annual_inc'] >= lowerc) & (loandata['annual_inc'] <= upperc)]


In [ ]:
# Next step is to Bin quantitive data for categorical analysis. First to be binned will be intrest rate.
print(loandata_up['int_rate'].max())
print(loandata_up['int_rate'].min())
#since the minimum data is at 5.42 and maximum is at 24.4 the bins will run from 5-25
loandata_up['int_bin'] = pd.cut(loandata_up['int_rate'], bins = [5.00,10.00,15.00,20.00,25.00], labels = ["5%-10%","11%-15%","16%-20%","21%-25%"])

In [ ]:
# Next is creating bin for the loan amount, from this we can see the loan run from 500 - 35000
print(loandata_up['loan_amnt'].describe())
binsloan= [0,5000,10000,15000,20000,25000,30000,35000] 
labelsloan = ["0-5k","5-10k","10-15k","15-20k","20-25k","25-30k","30-35k"]
loandata_up['loan_amnt_bin']=pd.cut(loandata_up['loan_amnt'],bins =binsloan, labels=labelsloan)

In [ ]:
# Next is creating bin for the annual income, from this we can see the loan run from 4000 - 145008
loandata_up['annual_inc'].describe()

In [ ]:
loandata_up['annual_incbin'] = pd.cut(loandata_up['annual_inc'], bins= [0, 10000, 20000, 40000, 60000,80000,100000, 120000,150000],
                                      labels=['0-10K','10-20K','20-40K','40-60K','60-80K','80-100K','100-120K','120-150K'])

loandata_up['annual_incbin'].value_counts()

## UniVariant Analysis
### lets start with analysing whats the percentage/count of loan status currently.

In [ ]:

counts = loandata_up['loan_status'].value_counts().rename_axis('loan_status').reset_index(name='count')
sns.set(rc={'figure.figsize':(12,9)})
plt.title('Loan Status', fontsize=20)
total = float(len(loandata_up))
ax = sns.barplot(x='loan_status', y='count', data=counts)
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

plt.show()

### It shows that 14.3% of the Loan status is Charged off

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='int_bin', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()



In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='loan_amnt_bin', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

In [ ]:

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='annual_incbin', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='grade', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

In [ ]:

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='emp_length', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

In [ ]:

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='home_ownership', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(35, 12))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='purpose', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
ax1.title.set_text('Purpose Percentage')
ax2.title.set_text('Purpose and Loan Status relationship')
plt.tight_layout()
plt.show()

In [ ]:
counts = loandata_up['addr_state'].value_counts().rename_axis('addr_state').reset_index(name='count')
sns.set(rc={'figure.figsize':(30,10)})
plt.title('State wise borrowing', fontsize=20)
total = float(len(loandata_up))
ax = sns.barplot(x='addr_state', y='count', data=counts)
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='center')

plt.show()

In [ ]:
#df_plot = loandata_up.groupby(['int_bin', 'loan_status']).size().reset_index().pivot(columns='int_bin', index='loan_status', values=0)

#df_plot.plot(kind='bar', stacked=True)


sns.displot(loandata_up, x='int_bin', hue='loan_status', multiple='stack')

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))
for ax, multiple in zip((ax1, ax2), ['layer', 'fill']):
    sns.histplot(data=loandata_up, x='int_bin', hue='loan_status', binwidth=10, stat='percent', multiple=multiple, ax=ax)
    ax.set_title(f"multiple='{multiple}'")

for bar_group, color in zip(ax.containers, ['black', 'white']):
    ax.bar_label(bar_group, label_type='center', color=color,
                 labels=[f'{bar.get_height() * 100:.1f} %' if bar.get_height() > 0 else '' for bar in bar_group])
ax.yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()